In [2]:
import numpy as np
import pandas as pd
from math import pi
import pandas as pd
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure,ColumnDataSource
from bokeh.transform import cumsum
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox,column
from bokeh.layouts import gridplot
from bokeh.models.widgets import Dropdown,Slider
from bokeh.models import Select,Panel
from bokeh.layouts import row
from bokeh.io import curdoc

In [3]:
def count_y_by_x(column,column_val,count_by,start_date,end_date,data)
    data_dates = data_date_interval(start_date,end_date,data)
    if data_dates.empty:
        x_list =[]
        y_list =[]
        return[x_list,y_list]
    column_by_x = data_dates.groupby([count_by,column]).size()
    y_by_x = column_by_x .loc[:,column_val].sort_values(ascending=False)
    y_list = y_by_x .tolist()[:12]
    y_by_x_df  = pd.DataFrame(y_by_x).reset_index()
    x_list = y_by_x_df[count_by].tolist()[:12]
    return [x_list,y_list]

In [4]:
def data_date_interval(start_date,end_date,data):
    data_dates = pd.DataFrame(columns=list(data.columns.values))
    for i in range(start_date,end_date+1):
        year = data[data['DATE_CASE'].str.contains(str(i))]
        data_dates = data_dates.append(year)
    return data_dates

In [5]:
def update_plot(attr, old, new):
    x = count_y_by_x(select_column_y.value,selecty.value,selectx.value,slider1.value,slider2.value,data)[0]
    y = count_y_by_x(select_column_y.value,selecty.value,selectx.value,slider1.value,slider2.value,data)[1]
    p.xaxis.axis_label = selectx.value
    p.yaxis.axis_label = selecty.value
    p.x_range.factors = x
    source.data = {'x' : x,'y' : y} 

In [6]:
def update_piechart(attr, old, new):
    data_dates = data_date_interval(slider1.value,slider2.value,data)
    data2 = data_dates.groupby(selectx.value).size().sort_values(ascending=False)[:15].reset_index(name='value2')
    z = data2['value2'].tolist()
    angle = (data2['value2']/data2['value2'].sum() * 2*pi).tolist()
    color = Category20c[len(data2)]
    index = data2.iloc[:,0].tolist()
    source2.data = {'z' : z,'index' : index,'angle' : angle,'color' : color} 

In [7]:
def update_dropdown(attr, old, new):
    selecty.options = data_dates[select_column_y.value].unique().tolist()
    selecty.value = data_dates[select_column_y.value].unique().tolist()[0]

In [8]:
data = pd.read_csv('500alerts.csv', sep=';', header=0, index_col = 0)
data = data.fillna(" ")

x = count_y_by_x('TYPE','feed','COUNT_ORIGEN',1970,2018,data)[0]
y = count_y_by_x('TYPE','feed','COUNT_ORIGEN',1970,2018,data)[1]

TOOLTIPS = [("Number of alerts", "@y")]

data_dates = pd.DataFrame(columns=list(data.columns.values))
data_dates = data

data2 = data.groupby('COUNT_ORIGEN').size().sort_values(ascending=False)[:15].reset_index(name='value2')

z = data2['value2'].tolist()
angle = (data2['value2']/data2['value2'].sum() * 2*pi).tolist()
color = Category20c[len(data2)]
index = data2.iloc[:,0].tolist()

source = ColumnDataSource(data={
    'x' : x,
    'y' : y,
})

source2 = ColumnDataSource(data={
    'z' : z,
    'index': index,
    'angle' : angle,
    'color' : color
})

selectx = Select(title="X AXIS", options=['CLASSIF','NOT_COUNTRY','PROD_CAT','TYPE','RISK_DECISION','ACTION_TAKEN','DISTRIBUTION_STAT','PRODUCT','COUNT_ORIGEN','COUNT_DESTIN','COUNT_CONCERN'], value='COUNT_ORIGEN')
selectx.on_change('value',update_plot)
selectx.on_change('value',update_piechart)

select_column_y = Select(title="Column where the y value is", options=['CLASSIF','NOT_COUNTRY','PROD_CAT','TYPE','RISK_DECISION','ACTION_TAKEN','DISTRIBUTION_STAT','PRODUCT','COUNT_ORIGEN','COUNT_DESTIN','COUNT_CONCERN'], value='TYPE')
select_column_y.on_change('value',update_dropdown)
select_column_y.on_change('value',update_plot)

selecty = Select(title="Y AXIS", options=['feed','food', 'FCM'], value='feed')
selecty.on_change('value',update_plot)

slider1 = Slider(start = 1970, end = 2018, step = 1, value = 1970, title =  'Start Date')
slider1.on_change('value',update_plot)
slider1.on_change('value',update_piechart)

slider2 = Slider(start = 1970, end = 2018, step = -1, value = 2018, title =  'End Date')
slider2.on_change('value',update_plot)
slider2.on_change('value',update_piechart)

#Figura 1
p = figure(x_range=x, plot_width=1300, plot_height=300, title="Alerts barchart", tooltips=TOOLTIPS, toolbar_location="left")
p.vbar(x ='x', top= 'y', width=0.9, source = source)
p.xgrid.grid_line_color = None
p.y_range.start = 0

#Figura 2
p2 = figure(plot_width=1000, plot_height=350, title="Productos con mas alertas",
        tools="hover", tooltips="@index: @z")

p2.wedge(x=0, y=1, radius=0.3,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='index', source=source2)

widgets = column(selectx, select_column_y, selecty,slider1,slider2)
main_row = row(widgets,p2)
series = column(p)

layout = column(main_row, series)

curdoc().add_root(layout)
curdoc().title = 'Alerts Dashboard'